In [1]:
'''! pip install lightning
! pip install torchmetrics
! pip install watermark
! pip install mlxtend
! pip install pandas'''

'! pip install lightning\n! pip install torchmetrics\n! pip install watermark\n! pip install mlxtend\n! pip install tensorboard\n! pip install pandas'

In [35]:
import time

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchvision import datasets, transforms

'''import lightning as L
from lightning.pytorch.loggers import WandbLogger, CSVLogger
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks import TQDMProgressBar
from lightning.pytorch.callbacks import ModelCheckpoint'''

import torchmetrics
import timm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from datamodules import RSNAdataset
from plotting import show_failures, plot_loss_and_acc

In [2]:
import wandb
wandb.login(key='a2a7828ed68b3cba08f2703971162138c680b664')

wandb: Currently logged in as: karanjot. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/karanjotvendal/.netrc


True

In [2]:
%reload_ext watermark
%watermark -a 'Karanjot Vendal' -v -p torch --iversion

Author: Karanjot Vendal

Python implementation: CPython
Python version       : 3.11.4
IPython version      : 8.14.0

torch: 2.0.1

lightning   : 2.0.6
timm        : 0.9.2
pandas      : 2.0.3
numpy       : 1.25.1
matplotlib  : 3.7.2
torchmetrics: 1.0.1
torchvision : 0.15.2
torch       : 2.0.1



# Hyperparameters

In [76]:
PATH = 'data/reduced_dataset/'
MODEL = 'resnet18'
BATCH_SIZE = 3
NUM_EPOCHS = 5
LEARNING_RATE = 0.0001

NUM_WORKERS = 0
NUM_CLASSES = 2

In [77]:
run = wandb.init(
      # Set the project where this run will be logged
      project="RACNet", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{1}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": LEARNING_RATE,
      "architecture": "CNN-GRU-MASK-FC-Classifier",
      "dataset": "MICAA MRI",
      "epochs": NUM_EPOCHS,
      "BATCH_SIZE": BATCH_SIZE
      })

NameError: name 'wandb' is not defined

# Model

In [78]:
class RACNet(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        
        self.cnn = timm.create_model(MODEL, pretrained=True, num_classes=0, in_chans=1)
        for param in self.cnn.parameters():
            param.requires_grad = False
        in_features = self.cnn(torch.randn(2, 1, 112, 112)).shape[1]
        
        self.rnn = nn.GRU(input_size=in_features, hidden_size=64, batch_first= True, bidirectional=False)
        
        self.fc = nn.Linear(16256, 32, bias=True)
        self.classifier = nn.Linear(32, num_classes, bias=True)

    def forward(self, x, org):
        # x shape: BxSxCxHxW
        batch_size, slices, C, H, W = x.size()
        c_in = x.view(batch_size * slices, C, H, W)
        #print('reshape input', c_in.shape)
        
        out = self.cnn(c_in)
        #print('CNN ouput', out.shape)
        
        rnn_in = out.view(batch_size, slices, -1)
        #print('reshaped rnn_in', rnn_in.shape)
        out, hd = self.rnn(rnn_in)
        #out =F.relu(self.RNN(out))

        #print('RNN ouput', out.shape)
        mask = self.mask_layer(org)
        out = out * mask
        #print('mask ouput', out.shape)
        
        batch, slices, rnn_features = out.size()
        out = out.reshape(batch_size, slices * rnn_features)
        #print('reshaped masked output', out.shape)
        
        out = F.relu(self.fc(out))
        #print('fc ouput', out.shape)
        
        logits = self.classifier(out)
        #print('classifier ouput', logits.shape)
        #output = F.softmax(logits, dim=1)
        #[prob 0, prob 1]

        return logits       

    def mask_layer(self, org):
        masks = []
        org = org[0].cpu().numpy()
        for i in org:
            print(i)
            dup = 254 - i
            mask_1 = torch.ones(i, 64) # .to(device='cuda')
            mask_0 = torch.zeros(dup, 64) #.to(device='cuda')
            mask = torch.cat((mask_1, mask_0), 0)
            masks.append(mask)

        masks = torch.stack(masks).to(device='cuda')
        return masks

# configuring the lightning module

In [79]:
#Configuring the lightning module
class LightningModel(L.LightningModule):
    def __init__(self, model, learning_rate, num_epochs, batch_size):
        super().__init__()

        self.learning_rate = learning_rate
        self.model = model
        self.num_epochs = num_epochs
        self.batch_size = batch_size

        self.save_hyperparameters(ignore=["model"])

        self.train_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        #self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=2)
        
        self.train_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2, average='macro')
        #self.val_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2, average='macro')
        self.test_f1 = torchmetrics.F1Score(task="multiclass", num_classes=2, average='macro')

        self.train_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)
        #self.val_auroc = AUROC(task="multiclass", num_classes=2)
        self.test_auroc = torchmetrics.AUROC(task="multiclass", num_classes=2)

    def forward(self, x, org):
        return self.model(x, org)

    def _shared_step(self, batch):
        dict = batch
        features = dict['X']
        print('input',features.shape)
        true_labels = dict['y']
        true_labels = true_labels.type(torch.cuda.LongTensor)
        print('targets',true_labels.shape)
        org = dict['org']
        print('org', type(org))
        out = self(features, org)

        loss = F.cross_entropy(out, true_labels)
        predicted_labels = torch.softmax(out, dim=1)
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("train_loss", loss)
        self.train_acc(predicted_labels, true_labels)
        self.log(
            "train_acc", self.train_acc, prog_bar=True, on_epoch=True, on_step=False
        )
        self.train_f1(predicted_labels, true_labels)
        self.log("train_f1", self.train_f1, prog_bar=True, on_epoch=True, on_step=False)

        self.train_auroc(predicted_labels, true_labels)
        self.log("train_auroc", self.train_auroc, prog_bar=True, on_epoch=True, on_step=False)
        return loss

    '''def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)

        self.log("val_loss", loss, prog_bar=False)
        self.val_acc(predicted_labels, true_labels)
        self.log("val_acc", self.val_acc, prog_bar=False)'''

    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc, on_epoch=True)
        
        sef.test_f1(predicted_labels, true_labels)
        self.log("train_f1", self.test_f1, prog_bar=True, on_epoch=True, on_step=False)

        self.test_auroc(predicted_labels, true_labels)
        self.log("train_auroc", self.test_auroc, prog_bar=True, on_epoch=True, on_step=False)
        
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

# Datamodule

In [80]:
class RSNADataModule(L.LightningDataModule):
    def __init__(self, fold, data_path=PATH, batch_size=10, num_workers=0, height_width=(112,112), mod="FLAIR"):
        super().__init__()
        self.batch_size = batch_size
        self.data_path = data_path
        self.height_width = height_width
        self.num_workers = num_workers
        self.fold = fold

    def prepare_data(self):
        
        self.train_transform = transforms.Compose(
            [
                transforms.Resize(self.height_width),
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
            ]
        )

        self.test_transform = transforms.Compose(
            [
                transforms.Resize(self.height_width),
                transforms.ToTensor(),
                transforms.Normalize((0.5), (0.5))
            ]
        )
        return

    def setup(self, stage=None):
        # Note transforms.ToTensor() scales input images
        # to 0-1 range
        folds_xtrain = np.load('./data/folds/xtrain.npy', allow_pickle=True)
        folds_xtest = np.load('./data/folds/xtest.npy', allow_pickle=True)
        folds_ytrain = np.load('./data/folds/ytrain.npy', allow_pickle=True)
        folds_ytest = np.load('./data/folds/ytest.npy', allow_pickle=True)
        
        xtrain = folds_xtrain[self.fold]
        ytrain = folds_ytrain[self.fold]
        xtest = folds_xtest[self.fold]
        ytest = folds_ytest[self.fold]

        
        self.train = RSNAdataset(
            self.data_path,
            xtrain,  
            ytrain,
            n_slices=254,
            img_size=112,
            transform=None
        )

        self.test = RSNAdataset(
            self.data_path,
            xtest,  
            ytest,
            n_slices=254,
            img_size=112,
            transform=None
        )

    
    def train_dataloader(self):
        train_loader = DataLoader(
            dataset=self.train,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=True,
            num_workers=self.num_workers,
        )
        return train_loader

    '''def val_dataloader(self):
        valid_loader = DataLoader(
            dataset=self.valid,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=False,
            num_workers=self.num_workers,
        )
        return valid_loader'''

    def test_dataloader(self):
        test_loader = DataLoader(
            dataset=self.test,
            batch_size=self.batch_size,
            drop_last=False,
            shuffle=False,
            num_workers=self.num_workers,
        )
        return test_loader

# preparing the dataset

In [81]:
dm = RSNADataModule(4, PATH, BATCH_SIZE, NUM_WORKERS)
dm.prepare_data()
dm.setup()

In [34]:
for batch in dm.train_dataloader():
    #train_counter.update(labels.tolist())
    #print('X',batch['X'])
    #print('y',batch['y'])
    print('org',batch['org'][0][1])
    #for i in batch['org']:
    #    print(i.items())
    break
    

org tensor(40)


# setting up trainer and logger

In [82]:
#config lighnting module
pymodel = RACNet(num_classes=NUM_CLASSES)
lightning_model = LightningModel(pymodel, learning_rate=LEARNING_RATE, batch_size=BATCH_SIZE, num_epochs=NUM_EPOCHS)

#checkpointing the best model
#configuring earlystopping
callbacks = [ModelCheckpoint(dirpath ='checkpoints/', 
                             filename= 'RACNet_fold{4}_f1:{train_f1}', 
                             save_top_k=1, mode='max', monitor='train_f1'),
            TQDMProgressBar(refresh_rate=50),
            EarlyStopping(monitor="train_loss", mode="min", patience=5)
            ]

#configuring logger
csv_logger = CSVLogger(save_dir='csv_logs/', name='ResNet18')
'''wandb_logger = WandbLogger(
      project="RACNet", 
      name=f"experiment_{2}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": LEARNING_RATE,
      "architecture": "CNN-LSTM",
      "dataset": "MICAA MRI",
      "epochs": NUM_EPOCHS,
      "batch size": BATCH_SIZE,
      "Image size": (112,112)
      })'''

'wandb_logger = WandbLogger(\n      project="RACNet", \n      name=f"experiment_{2}", \n      # Track hyperparameters and run metadata\n      config={\n      "learning_rate": LEARNING_RATE,\n      "architecture": "CNN-LSTM",\n      "dataset": "MICAA MRI",\n      "epochs": NUM_EPOCHS,\n      "batch size": BATCH_SIZE,\n      "Image size": (112,112)\n      })'

# Training

In [83]:
trainer = L.Trainer(
    max_epochs=NUM_EPOCHS,
    callbacks = callbacks,
    accelerator="gpu",
    devices=1,
    logger=[csv_logger],
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
start_time = time.time()

trainer.fit(model=lightning_model, datamodule=dm)
runtime = (time.time() - start_time)/60
print(f'Training finished in {runtime: .2f} min in total')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type               | Params
---------------------------------------------------
0 | model       | RACNet             | 11.8 M
1 | train_acc   | MulticlassAccuracy | 0     
2 | test_acc    | MulticlassAccuracy | 0     
3 | train_f1    | MulticlassF1Score  | 0     
4 | test_f1     | MulticlassF1Score  | 0     
5 | train_auroc | MulticlassAUROC    | 0     
6 | test_auroc  | MulticlassAUROC    | 0     
---------------------------------------------------
631 K     Trainable params
11.2 M    Non-trainable params
11.8 M    Total params
47.206    Total estimated model params size (MB)


Epoch 0:   0%|                                                                                  | 0/156 [00:00<?, ?it/s]input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
100
16
108
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
101
35
34
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
90
101
102
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
34
101
96
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
33
98
104
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
34
14
33
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
31
100
34
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
40
101
17
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
101
34
20


/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:42: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
/home/karanjotvendal/miniforge3/envs/thesis/lib/python3.11/site-packages/torchmetrics/utilities/prints.py:42: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028


input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
190
49
35
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
15
52
33
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
48
34
32
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
35
105
29
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
100
172
52
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
33
17
33
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
33
20
48
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
195
106
101
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
212
35
35
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <class 'list'>
34
35
15
input torch.Size([3, 254, 1, 112, 112])
targets torch.Size([3])
org <cla

/home/karanjotvendal/karanjot/thesis/RSNA/4 Task custom architecture/datamodules.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = [torch.tensor(image, dtype=torch.float32) for image in images]


# Evaluating the model

In [ ]:
#plotting loss and accuracy
plot_loss_and_acc(trainer.logger.log_dir)

In [ ]:
#Evaluating accuracy of Best(Checkpoint) model on test set
trainer.test(model=lightning_model, datamodule=dm, ckpt_path='best')

In [ ]:
#visualising cofusion matrix
from torchmetrics import ConfusionMatrix
import matplotlib
from mlxtend.plotting import plot_confusion_matrix

cmat = ConfusionMatrix(task='multiclass', num_classes=NUM_CLASSES)

for x, y in dm.test_dataloader():

    with torch.no_grad():
        pred = lightning_model(x)
    cmat(pred, y)

cmat_tensor = cmat.compute()
cmat = cmat_tensor.numpy()

fig, ax = plot_confusion_matrix(
    conf_mat=cmat,
    class_names=class_dict.values(),
    norm_colormap=matplotlib.colors.LogNorm()  
    # normed colormaps highlight the off-diagonals 
    # for high-accuracy models better
)

plt.show()